In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

import timeseriesclient as tsc

In [4]:
tsc.globalsettings.environment.set_qa()  # Work with test environment

# Create table
Lets create a table in Pandas -> DataFrame. 

In [5]:
table_dummy = pd.DataFrame({'temp': 20. + np.random.randn(100),
                           'strain': np.linspace(0.001, 0.01, num=100),
                           'stress': np.linspace(10.0, 359.0, num=100)})

In [6]:
table_dummy.head(10)

,strain,stress,temp
0,0.001000,10.000000,19.580254
1,0.001091,13.525253,19.110092
2,0.001182,17.050505,20.280230
3,0.001273,20.575758,20.353088
4,0.001364,24.101010,20.818352
5,0.001455,27.626263,19.236020
6,0.001545,31.151515,19.946317
7,0.001636,34.676768,18.861846
8,0.001727,38.202020,20.601047
9,0.001818,41.727273,19.211711


# Simple table uploader

In [7]:
def table_uploader(auth, table, table_name):
    client = tsc.TimeSeriesClient(auth)
    
    table_summary = {'name': table_name,
                     'columns': {}}
    for col in table:
        print col
        response = client.create(pd.DataFrame(table[col]))
        table_summary['columns'][col] = response['TimeSeriesId']
    return table_summary

# Upload to the reservoir

First, authenticate yourself towards the reservoir.

In [8]:
auth_key = tsc.Authenticator('reservoir-integrationtest@4subsea.com')

Password: ········


Then initiate a client and use the authentication key.

In [9]:
table_summary = table_uploader(auth_key, table_dummy, 'SuperDuperTable')

strain
stress
temp


Upload the DataFrame and store the response from the reservoir.

In [10]:
table_summary

{'columns': {'strain': u'c45733ed-70ec-4dd1-aa1b-2ec5736637f9',
  'stress': u'c27d34d5-b274-4c43-b602-660c28adc65c',
  'temp': u'87a9a5b0-107f-4c67-b467-3a9a49ebe707'},
 'name': 'SuperDuperTable'}

The reservoir assigns a unique id - TimeSeriesId. This id can be used to make requests towards the reservoir.

# Retrieve data back from the reservoir
You can download data from the reservoir. Data is presented as Pandas Series object. The index is by default kept as integers.

## Simple table downloader

In [11]:
def table_download(auth, table_summary):
    client = tsc.TimeSeriesClient(auth)
    table = {}
    
    for col in table_summary['columns']:
        print col
        table[col] = client.get(table_summary['columns'][col])
    df = pd.DataFrame(table)
    df.index.name = None
    return df

In [12]:
table = table_download(auth_key, table_summary)

strain
stress
temp


In [13]:
table.head(10)

,strain,stress,temp
0,0.001000,10.000000,19.580254
1,0.001091,13.525253,19.110092
2,0.001182,17.050505,20.280230
3,0.001273,20.575758,20.353088
4,0.001364,24.101010,20.818352
5,0.001455,27.626263,19.236020
6,0.001545,31.151515,19.946317
7,0.001636,34.676768,18.861846
8,0.001727,38.202020,20.601047
9,0.001818,41.727273,19.211711


# Delete data in the reservoir

In [19]:
def table_delete(auth, table_summary):
    client = tsc.TimeSeriesClient(auth)

    for col in table_summary['columns']:
        print col
        table[col] = client.delete(table_summary['columns'][col])